In [5]:
import pandas as pd

# Only needed to debug the direcroty. 
import os
cwd = os.getcwd()
print(cwd)

C:\Users\kenne\code\kennyrnwilson\git\health\analysis


# Calories and Macronutrients
We read in data exported from myfitnesspal

## Process the MyFitnessPal file

In [7]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# Read in the csv of weights
weight_df = pd.read_csv('..\data\my-fitness-pal\weight.csv', index_col = 'Date')

# Read in the csv of detailed nutritional data
all_nutrition_df = pd.read_csv('..\data\my-fitness-pal\macro-micro.csv')


# We are only interested in the calories and macro-nutrient amounts in grams
detailed_macros = all_nutrition_df[['Date','Calories','Fat (g)','Carbohydrates (g)','Protein (g)']]	

# The detailed_macros dataframe has a row per meal. The following logic groups and aggregates so we have 
# one row per day
aggregation_functions = {'Calories': 'sum', 'Fat (g)': 'sum', 'Protein (g)': 'sum', 'Carbohydrates (g)': 'sum'}
aggregated_macros = detailed_macros.groupby('Date').aggregate(aggregation_functions)
aggregated_macros

# I want to know the total calories from each macro
fat_cals = aggregated_macros['Fat (g)'] * 9
prot_cals = aggregated_macros['Protein (g)'] * 4
carb_cals = aggregated_macros['Carbohydrates (g)'] * 4

# Now get the percentages 
fat_percent = fat_cals / aggregated_macros['Calories']
prot_percent = prot_cals / aggregated_macros['Calories']
carb_percent = carb_cals / aggregated_macros['Calories']
fat_percent.name = 'Fat(%)'
carb_percent.name = 'Carb(%)'
prot_percent.name = 'Prot(%)'


final_data = pd.concat([weight_df['Weight'],aggregated_macros[['Calories','Protein (g)','Fat (g)','Carbohydrates (g)']], prot_percent, fat_percent, carb_percent], axis=1)
#print(final_data)
final_data.sort_index(inplace=True)
final_data.bfill(inplace=True)
final_data.ffill(inplace=True)
final_data



,Weight,Calories,Protein (g),Fat (g),Carbohydrates (g),Prot(%),Fat(%),Carb(%)
Date,,,,,,,,
2023-04-17,68.0,1920.1,127.8,48.6,228.0,0.266236,0.227801,0.474975
2023-04-18,68.0,2462.1,179.7,48.1,255.4,0.291946,0.175826,0.414930
2023-04-19,68.0,2164.7,140.1,68.1,234.3,0.258881,0.283134,0.432947
2023-04-20,68.0,2558.3,148.3,70.0,304.5,0.231873,0.246257,0.476097
2023-04-21,68.0,2191.0,199.1,30.9,279.5,0.363487,0.126928,0.510269
2023-04-22,67.8,2562.1,166.3,63.1,337.5,0.259631,0.221654,0.526912
2023-04-23,67.8,1093.2,80.2,38.4,104.0,0.293450,0.316136,0.380534
2023-04-24,67.2,1153.0,31.6,69.7,99.3,0.109627,0.544059,0.344493
2023-04-25,67.2,2671.4,135.2,95.2,312.1,0.202441,0.320731,0.467321


## Calculate Maintenance Calorie Statistics

In [21]:
# We want to use the last fourteen data points
rows = final_data.tail(14)

# Now we separate the foureen rows into week one and week two
week_one = last_fourteen.head(7)
week_two = last_fourteen.tail(7)

# Calculate the mean weight on each week
week_one_mean_wgt = week_one['Weight'].mean()
week_two_mean_wgt = week_two['Weight'].mean()

# Take the diffence of the two weeks as the average weight change per week
weekly_mean_wgt_change = week_two_mean_wgt - week_one_mean_wgt

# Convert weekly weight change in kilograms to weekly weight change in calories
weekly_mean_wgt_cals = weekly_mean_wgt_change * 3500

# Convert weekly weight change in caloreies to weekly daily weight change in calories
calorie_surplus = weekly_mean_wgt_cals / 7

# Calculate the average calorie consumption over the fourteen days
cals_mean = last_fourteen['Calories'].mean()



# Calculate estimated base calorie level
maint = cals_mean - calorie_surplus

print('Average weekly weight change in kg is ', weekly_mean_wgt_change)
print('Average daily weight change in kg is ', weekly_mean_wgt_change)
print('Average daily calorie consumption is ', cals_mean)
print('Daily calorie surplus is', calorie_surplus )
print('Estimated daily maintenance level is ', (maint))



Average weekly weight change in kg is  0.18571428571426907
Average daily weight change in kg is  0.18571428571426907
Average daily calorie consumption is  2545.057142857143
Daily calorie surplus is 92.85714285713455
Estimated daily maintenance level is  2452.2000000000085
